In [15]:
import sys
sys.path.append('../../')
from fourcastnet.model import *
from fourcastnet.utils import YParams
from torchinfo import summary

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Initialize FourCastNet model

In [10]:
cfg = YParams('../../fourcastnet/config/AFNO.yaml', 'full_field')

In [11]:
cfg.params.keys()

dict_keys(['loss', 'lr', 'scheduler', 'num_data_workers', 'dt', 'n_history', 'prediction_type', 'prediction_length', 'n_initial_conditions', 'ics_type', 'save_raw_forecasts', 'save_channel', 'masked_acc', 'maskpath', 'perturb', 'add_grid', 'N_grid_channels', 'gridtype', 'roll', 'max_epochs', 'batch_size', 'num_blocks', 'nettype', 'patch_size', 'width', 'modes', 'target', 'in_channels', 'out_channels', 'normalization', 'train_data_path', 'valid_data_path', 'inf_data_path', 'exp_dir', 'time_means_path', 'global_means_path', 'global_stds_path', 'orography', 'orography_path', 'log_to_screen', 'log_to_wandb', 'save_checkpoint', 'enable_nhwc', 'optimizer_type', 'crop_size_x', 'crop_size_y', 'two_step_training', 'plot_animations', 'add_noise', 'noise_std'])

In [13]:
model = AFNONet(
    **cfg.params
)

In [21]:
12443136 * 32 * 8 / 100000

31854.42816

In [18]:
summary(model, input_size=(1, 2, 720, 1440))

input shape: torch.Size([1, 2, 720, 1440])
post patch_embed shape: torch.Size([1, 16200, 768])
pre afno layers shape: torch.Size([1, 90, 180, 768])


Layer (type:depth-idx)                   Output Shape              Param #
AFNONet                                  [1, 2, 720, 1440]         12,443,136
├─PatchEmbed: 1-1                        [1, 16200, 768]           --
│    └─Conv2d: 2-1                       [1, 768, 90, 180]         99,072
├─Dropout: 1-2                           [1, 16200, 768]           --
├─ModuleList: 1-3                        --                        --
│    └─Block: 2-2                        [1, 90, 180, 768]         --
│    │    └─LayerNorm: 3-1               [1, 90, 180, 768]         1,536
│    │    └─AFNO2D: 3-2                  [1, 90, 180, 768]         297,984
│    │    └─LayerNorm: 3-3               [1, 90, 180, 768]         1,536
│    │    └─Mlp: 3-4                     [1, 90, 180, 768]         4,722,432
│    │    └─Identity: 3-5                [1, 90, 180, 768]         --
│    └─Block: 2-3                        [1, 90, 180, 768]         --
│    │    └─LayerNorm: 3-6               [1, 90, 180, 7

### Load pretrained weights

In [37]:
def load_model(model, checkpoint_file):
    model.zero_grad()
    checkpoint_fname = checkpoint_file
    checkpoint = torch.load(checkpoint_fname, map_location=torch.device('cpu'))

    try:
        new_state_dict = OrderedDict()
        for key, val in checkpoint['model_state'].items():
            name = key[7:]
            if name != 'ged':
                new_state_dict[name] = val  
        import pdb; pdb.set_trace()
        model.load_state_dict(new_state_dict)
    except:
        model.load_state_dict(checkpoint['model_state'])
    model.eval()
    return model

In [38]:
ckpt_path = '../../checkpoint/model_weights/FCN_weights_v0/backbone.ckpt'
model = load_model(model, ckpt_path)

RuntimeError: Error(s) in loading state_dict for AFNONet:
	Missing key(s) in state_dict: "pos_embed", "patch_embed.proj.weight", "patch_embed.proj.bias", "blocks.0.norm1.weight", "blocks.0.norm1.bias", "blocks.0.filter.w1", "blocks.0.filter.b1", "blocks.0.filter.w2", "blocks.0.filter.b2", "blocks.0.norm2.weight", "blocks.0.norm2.bias", "blocks.0.mlp.fc1.weight", "blocks.0.mlp.fc1.bias", "blocks.0.mlp.fc2.weight", "blocks.0.mlp.fc2.bias", "blocks.1.norm1.weight", "blocks.1.norm1.bias", "blocks.1.filter.w1", "blocks.1.filter.b1", "blocks.1.filter.w2", "blocks.1.filter.b2", "blocks.1.norm2.weight", "blocks.1.norm2.bias", "blocks.1.mlp.fc1.weight", "blocks.1.mlp.fc1.bias", "blocks.1.mlp.fc2.weight", "blocks.1.mlp.fc2.bias", "blocks.2.norm1.weight", "blocks.2.norm1.bias", "blocks.2.filter.w1", "blocks.2.filter.b1", "blocks.2.filter.w2", "blocks.2.filter.b2", "blocks.2.norm2.weight", "blocks.2.norm2.bias", "blocks.2.mlp.fc1.weight", "blocks.2.mlp.fc1.bias", "blocks.2.mlp.fc2.weight", "blocks

### Dummy inference pass